In [2]:
import psycopg2
import pandas as pd
import json


- Afficher pour chaque annexe, la liste des champs (./app/config.py)
- Pour chaque annexe, créer la requête pour récupérer les données concernant l'annexe de manière variabilisé
- Créer une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision. Cette fonction fait une requête à la base de donnée avec ces arguments. En retour elle renvoie un dataframe pandas 
- Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.
- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

# la liste des champs (./app/config.py) pour chaque annexe

In [2]:
# 1- Les données de toutes les annexes
import app.backend.config as conf
data = conf.CHAMPS_ANNEXES
data

{'DATA_PRES_CROISEE': ['RefFonc',
  'ChapitreNature',
  'Section',
  'InfoOpe',
  'CodRD',
  'OpBudg',
  'MtBudgPrec',
  'MtRARPrec',
  'MtPropNouv',
  'MtPrev',
  'CredOuv',
  'MtReal',
  'MtRAR3112',
  'MtCumul',
  'MtCumulCA',
  'MtMdtTitreEmis',
  'MtSup'],
 'DATA_EMPRUNT': ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv'

In [ ]:
for key, value in data.items() :
    print(key,value)

In [4]:
for item in data:
    print(item)

DATA_PRES_CROISEE
DATA_EMPRUNT
DATA_TRESORERIE
DATA_CHARGE
DATA_TIERS
DATA_CREDIT_BAIL
DATA_PPP
DATA_AUTRE_ENGAGEMENT
DATA_CONCOURS
DATA_RECETTE_AFFECTEE
DATA_FORMATION
DATA_FISCALITE
DATA_CONSOLIDATION
DATA_ORGANISME_ENG
DATA_ORGANISME_GROUP
DATA_PATRIMOINE
DATA_PERSONNEL
DATA_PERSONNEL_SOLDE
DATA_DETTE
DATA_VENTILATION
DATA_CONTRAT_COUV
DATA_AMORTISSEMENT_METHODE
DATA_PROVISION
DATA_APCP
DATA_SIGNATURE
DATA_SIGNATAIRE
DATA_ETAB_SERVICE
DATA_PRET
DATA_CONTRAT_COUV_REFERENCE
DATA_SERVICE_FERROVIAIRE_BUD
DATA_SERVICE_FERROVIAIRE_PATRIM
DATA_SERVICE_FERROVIAIRE_TER
DATA_FOND_COMM_HEBERGEMENT
DATA_FOND_EUROPEEN
DATA_FOND_EUROPEEN_PROGRAMMATION
DATA_FOND_AIDES_ECO
DATA_FORMATION_PRO_JEUNES
DATA_MEMBRESASA
DATA_FLUX_CROISES
DATA_SOMMAIRE


In [5]:
print(data.keys())

dict_keys(['DATA_PRES_CROISEE', 'DATA_EMPRUNT', 'DATA_TRESORERIE', 'DATA_CHARGE', 'DATA_TIERS', 'DATA_CREDIT_BAIL', 'DATA_PPP', 'DATA_AUTRE_ENGAGEMENT', 'DATA_CONCOURS', 'DATA_RECETTE_AFFECTEE', 'DATA_FORMATION', 'DATA_FISCALITE', 'DATA_CONSOLIDATION', 'DATA_ORGANISME_ENG', 'DATA_ORGANISME_GROUP', 'DATA_PATRIMOINE', 'DATA_PERSONNEL', 'DATA_PERSONNEL_SOLDE', 'DATA_DETTE', 'DATA_VENTILATION', 'DATA_CONTRAT_COUV', 'DATA_AMORTISSEMENT_METHODE', 'DATA_PROVISION', 'DATA_APCP', 'DATA_SIGNATURE', 'DATA_SIGNATAIRE', 'DATA_ETAB_SERVICE', 'DATA_PRET', 'DATA_CONTRAT_COUV_REFERENCE', 'DATA_SERVICE_FERROVIAIRE_BUD', 'DATA_SERVICE_FERROVIAIRE_PATRIM', 'DATA_SERVICE_FERROVIAIRE_TER', 'DATA_FOND_COMM_HEBERGEMENT', 'DATA_FOND_EUROPEEN', 'DATA_FOND_EUROPEEN_PROGRAMMATION', 'DATA_FOND_AIDES_ECO', 'DATA_FORMATION_PRO_JEUNES', 'DATA_MEMBRESASA', 'DATA_FLUX_CROISES', 'DATA_SOMMAIRE'])


# 2 - Récupérer les données concernant l'annexe de manière variabilisé

In [3]:

def annexe(nom):
    donnees = data[nom]
    return donnees

In [4]:
# 2.1 - Afficher les donnees de DATA_EMPRUNT
annexe(nom="DATA_EMPRUNT")

['AnEncaisse',
 'AnnuitNetDette',
 'Champ_Editeur',
 'CodArticle',
 'CodNatEmpr',
 'CodPeriodRemb',
 'CodPeriodRembDtVote',
 'CodPeriodRembReneg',
 'CodProfilAmort',
 'CodProfilAmortDtVote',
 'CodProfilAmortReneg',
 'CodTypEmpr',
 'CodTypEmprGaranti',
 'CodTypPreteur',
 'CodTypTxCouv',
 'CodTypTxReneg',
 'CodTyptxDtVote',
 'CodTyptxInit',
 'CoutSortie',
 'Couverture',
 'Dt1RembInit',
 'DtDebCouv',
 'DtEmission',
 'DtFinContr',
 'DtFinCouv',
 'DtPeriodeBonif',
 'DtRegltCouv',
 'DtReneg',
 'DtSignInit',
 'DureeAnn',
 'DureeContratInit',
 'DureeContratReneg',
 'DureeRest',
 'DureeRestInit',
 'DureeRestReneg',
 'IndSousJacent',
 'IndSousJacentApresCouv',
 'IndSousJacentAvantCouv',
 'IndSousJacentDtVote',
 'IndexTxVariDtVote',
 'IndexTxVariInit',
 'IndexTxVariReneg',
 'IndiceCouv',
 'IndiceEmpr',
 'IndiceEmprDtVote',
 'LibCpte',
 'LibOrgCoContr',
 'LibOrgaPreteur',
 'MPrimeRecueCouv',
 'MtCRDCouvert',
 'MtCRDRefin',
 'MtCapitalExer',
 'MtCapitalReamenage',
 'MtCapitalRestDu_01_01',
 'MtCapi

In [6]:
# 2.2 - Afficher les donnees de toutes les annexes
for i in data:
    donnee = annexe(nom=i)
    print(donnee)

['RefFonc', 'ChapitreNature', 'Section', 'InfoOpe', 'CodRD', 'OpBudg', 'MtBudgPrec', 'MtRARPrec', 'MtPropNouv', 'MtPrev', 'CredOuv', 'MtReal', 'MtRAR3112', 'MtCumul', 'MtCumulCA', 'MtMdtTitreEmis', 'MtSup']
['AnEncaisse', 'AnnuitNetDette', 'Champ_Editeur', 'CodArticle', 'CodNatEmpr', 'CodPeriodRemb', 'CodPeriodRembDtVote', 'CodPeriodRembReneg', 'CodProfilAmort', 'CodProfilAmortDtVote', 'CodProfilAmortReneg', 'CodTypEmpr', 'CodTypEmprGaranti', 'CodTypPreteur', 'CodTypTxCouv', 'CodTypTxReneg', 'CodTyptxDtVote', 'CodTyptxInit', 'CoutSortie', 'Couverture', 'Dt1RembInit', 'DtDebCouv', 'DtEmission', 'DtFinContr', 'DtFinCouv', 'DtPeriodeBonif', 'DtRegltCouv', 'DtReneg', 'DtSignInit', 'DureeAnn', 'DureeContratInit', 'DureeContratReneg', 'DureeRest', 'DureeRestInit', 'DureeRestReneg', 'IndSousJacent', 'IndSousJacentApresCouv', 'IndSousJacentAvantCouv', 'IndSousJacentDtVote', 'IndexTxVariDtVote', 'IndexTxVariInit', 'IndexTxVariReneg', 'IndiceCouv', 'IndiceEmpr', 'IndiceEmprDtVote', 'LibCpte', 'L

# Création d'une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision

In [3]:
# 3- connexion à la base de donnée et  retour d'un dataframe pandas 
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")


def get_data(connexion):
    query= """SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = 'DATA_EMPRUNT'
    AND documentbudgetaire.exercice = '2020'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 3 """
    dat = pd.read_sql_query(query, conn)
    # conn.close()
    return dat
get_data(connexion=conn)

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,341352,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""8102.60""}]",59318,21240438800044,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",59318,LOGEMENTS,24438,M14-M14_COM_INF500,...,None,1,A,21240438800010,"[{""Nature"": ""6541"", ""ContNat"": ""65"", ""ArtSpe"":...",21240438800044,21240438800044,COMMUNE DE ST-LAURENT-LA-VALLEE,Commune,24
1,341390,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",59326,21520338100011,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",59326,Communes de Silvarouvres,52474,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6558"", ""ContNat"": ""65"", ""ArtSpe"":...",21520338100011,21520338100011,Commune de Silvarouvres,Commune,52
2,341406,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",59329,24170043400020,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",59329,BUDGET PRINCIPAL,17300,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""020"", ""Fonction"": ""01"", ""ContNat""...",24170043400020,24170043400020,COMM. D'AGGLOMÉRATION DE LA ROCHELLE,EPCI,None


In [7]:
def connection_pg2():
    return psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

with connection_pg2() as conn:
    dat = get_data(conn)

dat.head(2)

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,325931,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""5437.00""}]",56630,26820284300010,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",56630,BUDGET CCAS,82179,M14-M14_CCAS_INF3500,...,None,1,P,None,"[{""Nature"": ""6182"", ""ContNat"": ""011"", ""ArtSpe""...",26820284300010,26820284300010,COMMUNE DE SEPTFONDS,COMMUNE,82
1,325952,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""381708.74""}]",56634,21270192400012,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",56634,Budget communal,27192,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6225"", ""ContNat"": ""011"", ""ArtSpe""...",21270192400012,21270192400012,CROSVILLE LA VIEILLE,COMMUNE,27


In [5]:
df_emprunt = get_data(connexion=conn)

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
df_emprunt

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,341352,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""8102.60""}]",59318,21240438800044,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",59318,LOGEMENTS,24438,M14-M14_COM_INF500,...,None,1,A,21240438800010,"[{""Nature"": ""6541"", ""ContNat"": ""65"", ""ArtSpe"":...",21240438800044,21240438800044,COMMUNE DE ST-LAURENT-LA-VALLEE,Commune,24
1,341390,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",59326,21520338100011,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",59326,Communes de Silvarouvres,52474,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6558"", ""ContNat"": ""65"", ""ArtSpe"":...",21520338100011,21520338100011,Commune de Silvarouvres,Commune,52
2,341406,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",59329,24170043400020,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",59329,BUDGET PRINCIPAL,17300,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""020"", ""Fonction"": ""01"", ""ContNat""...",24170043400020,24170043400020,COMM. D'AGGLOMÉRATION DE LA ROCHELLE,EPCI,None


In [8]:
df_emprunt.json_annexe

0        [{"CodTypEmpr": "10", "RReelFon": "8102.60"}]
1    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
2    [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
Name: json_annexe, dtype: object

In [7]:
# 3.1 -
list(df_emprunt.columns)

['id',
 'type_annexe',
 'json_annexe',
 'fk_id_document_budgetaire',
 'siret_etablissement',
 'list_annexes',
 'id',
 'libelle',
 'code_insee',
 'nomenclature',
 'exercice',
 'nature_dec',
 'num_dec',
 'nature_vote',
 'type_budget',
 'id_etabl_princ',
 'json_budget',
 'fk_siret_collectivite',
 'siret_coll',
 'libelle_collectivite',
 'nature_collectivite',
 'departement']

0    09
1    09
2    09
Name: nature_dec, dtype: object

In [7]:
# 3.2 -
df_emprunt.json_annexe

0        [{"CodTypEmpr": "10", "RReelFon": "8102.60"}]
1    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
2    [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
Name: json_annexe, dtype: object

In [8]:
# 3.3 -
for item in df_emprunt.json_annexe:
    print(type(item))

<class 'str'>
<class 'str'>
<class 'str'>


In [1]:
# 3.4 - avec f string de manière variabiliser
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, nature_dec):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{nature_dec}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, nature_dec= '09')

NameError: name 'psycopg2' is not defined

# Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.

In [17]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace          

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [18]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
print(next(replace_natdec))
print(next(replace_natdec))

('01', 'Budget primitif')
('02', 'Décision modificative')


In [19]:
list(df_emprunt.columns)

['id',
 'type_annexe',
 'json_annexe',
 'fk_id_document_budgetaire',
 'siret_etablissement',
 'list_annexes',
 'id',
 'libelle',
 'code_insee',
 'nomenclature',
 'exercice',
 'nature_dec',
 'num_dec',
 'nature_vote',
 'type_budget',
 'id_etabl_princ',
 'json_budget',
 'fk_siret_collectivite',
 'siret_coll',
 'libelle_collectivite',
 'nature_collectivite',
 'departement']

In [20]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(df_emprunt.columns) if item not in interesting_columns]
df = df_emprunt.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
df_min
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""2197712.41""}]",20006675100021,BA OM,M14-M14_COM_SUP3500,2020,Compte administratif,3,Annexe,20006675100021,CC XAINTRIE VAL DORDOGNE,Communauté de commune à FPU
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21310429200015,budget Communal,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21310429200015,POMPERTUZAT,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21350103400010,COMMUNE EANCE - 290,M14-M14_COM_INF500,2020,Compte administratif,1,Principal,21350103400010,COMMUNE EANCE,Commune
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21440164800016,COMMUNE ST HILAIRE DE CHALEONS COMM,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21440164800016,COMMUNE ST HILAIRE DE CHALEONS,Commune


In [21]:
df_min.json_annexe

0    [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
1     [{"CodTypEmpr": "10", "RReelFon": "2197712.41"}]
2    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
3    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
4    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
Name: json_annexe, dtype: object

In [8]:
EMPRUNT_FIELD = annexe(nom="DATA_EMPRUNT")


NameError: name 'annexe' is not defined

In [9]:
df.json_annexe

NameError: name 'df' is not defined

In [24]:
# extraction de la colonne json_annexe
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))

json_df.assign(**row)
#print('dfs')
#print(dfs)
df_to_csv = pd.concat(dfs)
df_to_csv

,CodTypEmpr,CodProfilAmort,CodProfilAmortDtVote,CodArticle,LibCpte,AnEncaisse,ObjEmpr,MtEmprOrig,DureeRest,CodTypPreteur,...,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,01,C,C,167,Emprunts et dettes assorties de conditions par...,2009,Réseau 2e T Avance Agence de l'Eau,40300,4,02,...,21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
1,01,C,C,1641,Emprunts en euros,2009,Réseau 2ème T,85000,4,01,...,21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
2,01,C,C,1641,Emprunts en euros,2012,EXTENSION 1ère T BOURG+ARMOIRE ELEC,26000,2,01,...,21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
3,01,C,C,167,Emprunts et dettes assorties de conditions par...,2013,AVANCE COMMUNE,7200,0,02,...,21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
4,01,C,C,1641,Emprunts en euros,2014,AVANCE COMMUNE,14000,0,02,...,21240564100045,SERVICE ASSAINISSEMENT MAIRIE DE VANXAINS,M4-M49_A,2020,Compte administratif,1,Annexe,21240564100045,MAIRIE DE VANXAINS,Services de distribution d'eau potable et assa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,01,C,C,1641,None,2019,None,2000000.00,13.14,01,...,24770033900010,BUDGET PRINCIPAL,M14-M14_COM_SUP3500,2020,Compte administratif,2,Principal,24770033900010,VAL D'EUROPE AGGLOMERATION,CA VAL D'EUROPE dont la population est de 3500...
251,01,P,P,1641,None,2020,None,6000000.00,20.01,01,...,24770033900010,BUDGET PRINCIPAL,M14-M14_COM_SUP3500,2020,Compte administratif,2,Principal,24770033900010,VAL D'EUROPE AGGLOMERATION,CA VAL D'EUROPE dont la population est de 3500...
252,01,C,C,1641,None,2021,None,5000000.00,20.17,02,...,24770033900010,BUDGET PRINCIPAL,M14-M14_COM_SUP3500,2020,Compte administratif,2,Principal,24770033900010,VAL D'EUROPE AGGLOMERATION,CA VAL D'EUROPE dont la population est de 3500...
0,01,P,P,1641,None,2015,ETUDE DIAGNOSTIQUE REAU EAU,30000.00,0.00,01,...,21110289200034,SERVICE EAU DE PIEUSSE,M4-M49_A,2020,Compte administratif,1,Principal,21110289200034,SERVICE EAU DE PIEUSSE,


- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [25]:
def generate_csv(df, name):
    csv_field = df.to_csv(name)
    return csv_field
generate_csv(df_emprunt, "emprunt.csv")

In [26]:
# 3.4 - avec f string
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = , host = "ab-postgres",
                        port = "5432")"vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd"

def get_data(connexion, annexe, year, numero_docbudge):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{numero_docbudge}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, numero_docbudge= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,292380,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",50831,20005560600012,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FISCALITE, ...",50831,COMMUNE LES SEPTVALLONS,02439,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""615231"", ""ContNat"": ""011"", ""ArtSp...",20005560600012,20005560600012,COMMUNE LES SEPTVALLONS,COMMUNE,02
1,292447,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",50842,21470157500208,"[DATA_EMPRUNT, DATA_CHARGE, DATA_CONSOLIDATION...",50842,PARKING SOUTERRAIN MARMANDE,47157,M4-M4,...,None,1,A,21470157500018,"[{""Nature"": ""6811"", ""ContNat"": ""042"", ""ArtSpe""...",21470157500208,21470157500208,MARMANDE,VILLE,047
2,292492,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",50847,21560121200016,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_AUTRE_ENG...",50847,BUDGET PRINCIPAL,56121000000,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""2051"", ""Fonction"": ""321"", ""ContNa...",21560121200016,21560121200016,VILLE DE LORIENT,COMMUNE,None
3,293265,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",50986,25540201800014,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",50986,Syndicat des Eaux/Champey,54876,M4-M49_A,...,None,1,P,None,"[{""Nature"": ""001"", ""ArtSpe"": ""false"", ""CodRD"":...",25540201800014,25540201800014,Syndicat des Eaux et d'Assainissement/Champey,Syndicat des Eaux/Champey,54
4,293531,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",51025,21440196000072,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_SIGNATURE,...",51025,COMMERCE DE PROXIMITE,None,M14-M14_COM_500_3500,...,None,1,A,21440196000015,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",21440196000072,21440196000072,COMMERCE DE PROXIMITE,COMMUNE,None


In [ ]:
annexe_dic = dict()
for item in data:
    annexe_dic[item] =  get_data(connexion=conn, annexe=item, year=2020, numero_docbudge= '09')

In [28]:
annexe_dic["DATA_EMPRUNT"]

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,293441,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",51010,21520015500012,"[DATA_EMPRUNT, DATA_SIGNATURE, DATA_SIGNATAIRE...",51010,COMMUNE D'ARNANCOURT (106-00),52019,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ArtSpe"": ""false"", ""CodRD"":...",21520015500012,21520015500012,COMMUNE D'ARNANCOURT (106-00),COMMUNE D'ARNANCOURT (106-00),52
1,293482,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",51019,21900080900018,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",51019,COMMUNE DE PHAFFANS,90080,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""1641"", ""ContNat"": ""16"", ""ArtSpe"":...",21900080900018,21900080900018,COMMUNE DE PHAFFANS,COMMUNE,90
2,293496,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""TotGarEchoirExer"": ""339...",51021,21890345800014,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_TIERS, DA...",51021,COMMUNE DE SAINT-FLORENTIN,89345,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""615232"", ""Fonction"": ""8"", ""ContNa...",21890345800014,21890345800014,COLLECTIVITE SAINT-FLORENTIN,Commune,None
3,293547,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",51029,21120061300043,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",51029,BUDGET EAU ET ASSAINISSEMENT,12061,M4-M49_A,...,None,1,A,211200613,"[{""Nature"": ""605"", ""ContNat"": ""011"", ""ArtSpe"":...",21120061300043,21120061300043,CASTELNAU DE MANDAILLES,COMMUNE,12
4,293571,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",51032,21840067900013,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",51032,Mairie de Loriol-du-Comtat,84067,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""60611"", ""LibCpte"": ""EAU ET ASSAIN...",21840067900013,21840067900013,COMMUNE DE LORIOL DU COMTAT,Commune,84


In [29]:
for item in annexe_dic:
    print(annexe_dic[item])

Empty DataFrame
Columns: [id, type_annexe, json_annexe, fk_id_document_budgetaire, siret_etablissement, list_annexes, id, libelle, code_insee, nomenclature, exercice, nature_dec, num_dec, nature_vote, type_budget, id_etabl_princ, json_budget, fk_siret_collectivite, siret_coll, libelle_collectivite, nature_collectivite, departement]
Index: []

[0 rows x 22 columns]
       id   type_annexe                                        json_annexe  \
0  293441  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...   
1  293482  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...   
2  293496  DATA_EMPRUNT  [{"CodTypEmpr": "10", "TotGarEchoirExer": "339...   
3  293547  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...   
4  293571  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...   

   fk_id_document_budgetaire siret_etablissement  \
0                      51010      21520015500012   
1                      51019      21900080900018   
2       

In [ ]:
for  annex_name in data:
    get_data(connexion=conn, annexe= annex_name, year= 2020, numero_docbudge = '09')
    generate_csv(df=dat, name=f'{annex_name}.csv')

In [38]:
for  annex_name in data:
    get_data(connexion=conn, annexe= annex_name, year= 2020, numero_docbudge = '09')
    interesting_columns = []
    to_drop = [item for item in list(f'{annex_name}'.columns) if item not in interesting_columns]
    df = annex_name.drop(to_drop, axis=1)
    df.replace(dict_replace, inplace=True)
    df_min = df.head(5)
    df_min
    generate_csv(df=dat, name=f'{annex_name}.csv')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


InterfaceError: connection already closed

In [31]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire 
     ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite 
     ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 LIMIT 100
 """
dat = pd.read_sql_query(query, conn)
conn.close()



/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [32]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [33]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

In [34]:
df_min

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""2898.00""}]",26430246400013,CCAS DE SAINT-VIDAL,M14-M14_CCAS_INF3500,2020,Compte administratif,1,Principal,26430246400013,SAINT-VIDAL,Commune
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21360082800015,PRINCIPAL,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21360082800015,GEHEE,Comptabilité des communes de - 3500 habitants
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21100280300016,Commune de Plaines St Lange,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21100280300016,COMMUNE DE PLAINES SAINT LANGE,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21540583800016,Principal,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21540583800016,Commune de Villey-le-Sec,Commune de Villey-le-Sec
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21740290800016,Budget Principal,M14-M14_COM_INF500,2020,Compte administratif,1,Principal,21740290800016,COMMUNE DE VALLORCINE,COMMUNE


In [35]:
dfs = []
#EMPRUNT_FIELD = interesting_columns
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    print(json_df)
    #json_df.assign(**row)

  CodTypEmpr RReelFon AnEncaisse AnnuitNetDette Champ_Editeur CodArticle  \
0         10  2898.00       None           None          None       None   

  CodNatEmpr CodPeriodRemb CodPeriodRembDtVote CodPeriodRembReneg  ...  \
0       None          None                None               None  ...   

  TxActuaReneg TxApresCouv TxMargeInit TxMaxi TxMini TxPaye TxRecu Txinit  \
0         None        None        None   None   None   None   None   None   

  TypCouv TypeSortie  
0    None       None  

[1 rows x 97 columns]
  CodTypEmpr CodProfilAmort CodProfilAmortDtVote CodArticle AnEncaisse  \
0         01              X                    X       1641       2012   
1         01              X                    X       1641       2016   

                                          ObjEmpr MtEmprOrig DureeRest  \
0                          TRANSFERT DE LA MAIRIE      40000      2.00   
1  REABILITATION LOGEMENT 3 BIS PLACE DE L'EGLISE      30000      6.00   

  DureeRestInit CodTypPreteu

In [36]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('./csv/2020-CA-Limit100.csv', index=False)  

In [37]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']